### Import Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from scipy.stats import mode
from scipy.io import loadmat
from sklearn.cluster import KMeans
import random

<div dir="rtl">
    کتابخانه‌های مورد نیاز برای اجرای الگوریتم‌های خوشه‌بندی و ارزیابی عملکرد آنها بارگذاری می‌شوند. این کتابخانه‌ها شامل pandas و numpy برای کار با داده‌ها، sklearn برای استفاده از مدل‌ها و ارزیابی عملکرد آنها، scipy برای استفاده از توابع آماری و بارگذاری دیتاست، و random برای استفاده از توابع مربوط به تولید اعداد تصادفی است.
</div>


### Load and Prepare the Dataset

In [2]:
X = pd.read_csv("breast_data.csv")
print(X.shape)

breast_labels = pd.read_csv('breast_labels.csv')
print( breast_labels.shape)

if breast_labels.shape[0] == X.shape[0]:
    y_true = breast_labels 
else:
    print("Number of rows in breast_labels does not match the number of rows in X.")


(568, 30)
(568, 1)


<div dir="rtl">
    دو مجموعه داده از فایل‌های csv بارگذاری می‌شوند. ابعاد هر یک از مجموعه‌های داده چاپ می‌شود تا مطمئن شویم که بارگذاری درست انجام شده است.
</div>


### Implement the K-Means Clustering Algorithm

In [3]:
def kmeanscluster(X, k, mu, tol=1e-4, maxIter=300):
    n_samples, n_features = X.shape
    C = np.zeros(n_samples, dtype=int)
    
    for i in range(maxIter):
        for j in range(n_samples):
            distances = np.linalg.norm(X.iloc[j].values - mu, axis=1)
            C[j] = np.argmin(distances)
        
        mu_old = mu.copy()
        for j in range(k):
            if np.any(C == j):
                mu[j] = X[C == j].mean(axis=0).values
        
        if np.all(C == np.argmin([np.linalg.norm(X.iloc[j].values - mu, axis=1) for j in range(n_samples)], axis=1)):
            break
        
        if np.linalg.norm(mu - mu_old) < tol:
            break
            
    return C, mu

<div dir="rtl">
    تابع "kmeanscluster"، الگوریتم خوشه‌بندی k-means را برای داده‌های ورودی اجرا می‌کند. ورودی‌های این تابع شامل ماتریس داده‌ها (X)، تعداد خوشه‌های مورد نظر (k)، مقدار اولیه‌های مرکز خوشه‌ها (mu)، مقدار مجاز برای تغییرات کوچک در مراکز خوشه‌ها (tol)، و حداکثر تعداد تکرارهای مجاز (maxIter) است.
</div>


In [4]:
k = 2
accuracies = []
for i in range(5):   
    np.random.seed(i)
    initial_centroids = X.sample(n=k, random_state=i).values

    C, mu = kmeanscluster(X, k, initial_centroids)

    C = C.astype(int)
    accuracy1 = accuracy_score(y_true, C)
    accuracy2 = accuracy_score(y_true, 1 - C)
    
    accuracy = max(accuracy1, accuracy2)
    accuracies.append(accuracy)
    print(f"Run {i+1}, Accuracy: {accuracy}")

print("Accuracies for 5 runs with different starting points:", accuracies)

Run 1, Accuracy: 0.8538732394366197
Run 2, Accuracy: 0.8538732394366197
Run 3, Accuracy: 0.8538732394366197
Run 4, Accuracy: 0.8538732394366197
Run 5, Accuracy: 0.8538732394366197
Accuracies for 5 runs with different starting points: [0.8538732394366197, 0.8538732394366197, 0.8538732394366197, 0.8538732394366197, 0.8538732394366197]


<div dir="rtl">
    یک حلقه برای اجرای الگوریتم k-means به صورت تکراری بر روی داده‌های ورودی (X) و برچسب‌های واقعی (y_true) ایجاد شده است. برای هر تکرار، مراکز اولیه برای خوشه‌ها به صورت تصادفی انتخاب می‌شوند. سپس الگوریتم k-means با استفاده از تابع "kmeanscluster" اجرا می‌شود. پس از اتمام اجرا، برچسب‌های خوشه‌بندی‌شده (C) از خروجی الگوریتم بدست می‌آید. برای اندازه‌گیری دقت، دو مقدار دقت محاسبه می‌شود: یک مقدار با فرضیه اینکه مقادیر برچسب‌های خوشه‌بندی شده مطابق با برچسب‌های واقعی است و یک مقدار با فرضیه اینکه مقادیر برچسب‌های خوشه‌بندی شده معکوس برچسب‌های واقعی است. در پنج بار اجرای الگوریتم K-Means با نقاط شروع مختلف، ممکن است دقت‌های بدست آمده متفاوت باشند. دقت میانگین به دست آمده نشان‌دهنده کارایی مناسب الگوریتم K-Means در خوشه‌بندی داده‌های سرطان است.
</div>


In [5]:
initial_centroids.shape

(2, 30)

### Clustering with Provided Initial Centers Scope

In [6]:
init_mu = loadmat('init_mu.mat')['mu_init'].T 
k = init_mu.shape[0]  

C, _ = kmeanscluster(X, k, init_mu)

C = C.astype(int)
accuracy1 = accuracy_score(y_true, C)
accuracy2 = accuracy_score(y_true, 1 - C)

accuracy = max(accuracy1, accuracy2)
print("Accuracy of the clustering:", accuracy)



Accuracy of the clustering: 0.8538732394366197


<div dir="rtl">
    ماتریس مراکز اولیه برای الگوریتم k-means از یک فایل با پسوند .mat با استفاده از تابع "loadmat" خوانده شده و سپس تعداد خوشه‌ها از اندازه این ماتریس مشخص شده و الگوریتم k-means با استفاده از ماتریس مراکز اولیه ورودی اجرا می‌شود.
</div>


In [7]:
from sklearn.metrics import silhouette_score

C, _ = kmeanscluster(X, k, init_mu)
C = C.astype(int)

silhouette_avg = silhouette_score(X, C)

print("Silhouette Score:", silhouette_avg)

Silhouette Score: 0.6972648853324666


<div dir="rtl">
در کلاسترینگ، اغلب از معیارهایی مانند silhouette score استفاده می‌شود زیرا این معیارها اطلاعات بیشتری از ساختار داده‌ها و توزیع خوشه‌ها ارائه می‌دهند نسبت به دقت ساده. اگر میانگین silhouette score برای همه داده‌ها نزدیک به ۱ باشد، نشان می‌دهد که داده‌های هر خوشه بهتر از داده‌های خوشه‌های دیگر با هم مطابقت دارند و خوشه‌بندی مناسبی انجام شده است. در کلاسترینگ، معمولاً دقت به تنهایی معیار مناسبی نیست. زیرا معمولاً در کلاسترینگ، اطلاعات برچسب‌های واقعی موجود نیست و بنابراین نمی‌توان از مقایسه برچسب‌ها برای ارزیابی کیفیت خوشه‌بندی استفاده کرد. 

</div>


<div dir="rtl">
    با استفاده از روش‌های دیگر یادگیری بدون نظارت و همچنین یادگیری با نظارت می‌توان به دقت متفاوتی دست یافت.
</div>

Unsupervised Learning Models: k-Means Clustering, Hierarchical Clustering, DBSCAN (Density-Based Spatial Clustering of Applications with Noise), Gaussian Mixture Models (GMM), Principal Component Analysis (PCA), Autoencoders, Isolation Forest

Supervised Learning Models: Linear Regression, Logistic Regression, Decision Trees, Random Forest, Support Vector Machines (SVM), k-Nearest Neighbors (k-NN), Neural Networks, Gradient Boosting Machines (GBM)

### Alternative Unsupervised Learning Method: DBSCAN

In [10]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import accuracy_score

dbscan = DBSCAN(eps=0.5, min_samples=5)

dbscan.fit(X)

dbscan_labels = dbscan.labels_

dbscan_labels_mapped = np.where(dbscan_labels == -1, 2, dbscan_labels)

dbscan_accuracy = accuracy_score(y_true, dbscan_labels_mapped)
print("DBSCAN Accuracy:", dbscan_accuracy)


DBSCAN Accuracy: 0.0


<div dir="rtl">
    ابتدا الگوریتم DBSCAN با استفاده از کلاس "DBSCAN" از ماژول "sklearn.cluster" ایجاد می‌شود. سپس، الگوریتم بر روی داده‌های ورودی (X) اجرا می‌شود و برچسب‌های نهایی برای هر نمونه در متغیر "dbscan_labels" ذخیره می‌شوند. مقدار صفر بودن دقت الگوریتم DBSCAN معمولاً نشان‌دهنده عدم توانایی الگوریتم در تشخیص الگوهای خوشه‌بندی مناسب در داده‌ها است. 
</div>


### Alternative Supervised Learning Method: Random Forest Classifier

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_true, test_size=0.2, random_state=42)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

rf_classifier.fit(X_train, y_train)

rf_predictions = rf_classifier.predict(X_test)

rf_accuracy = accuracy_score(y_test, rf_predictions)
print("Random Forest Classifier Accuracy:", rf_accuracy)


c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Random Forest Classifier Accuracy: 0.9736842105263158


<div dir="rtl">
    الگوریتم RandomForestClassifier با 100 درخت تصادفی آموزش داده می‌شود و سپس بر روی داده‌های آزمایشی استفاده می‌شود تا پیش‌بینی انجام دهد. در نهایت، با استفاده از معیار دقت، عملکرد مدل RandomForestClassifier ارزیابی می‌شود و دقت پیش‌بینی چاپ می‌شود.

</div>


<div dir="rtl">
    اگر الگوریتم K-Means با مراکز واقعی که پس از خوشه‌بندی واقعی به دست آمده‌اند، مقداردهی اولیه شود، نتایج بهبود می‌یابد. به عبارت دیگر، مشاهده می‌شود که الگوریتم K-Means سریع‌تر به جواب بهینه همگرا می‌شود و خوشه‌بندی دقیق‌تری ارائه می‌شود. دلیل این بهبود، نزدیک بودن مراکز واقعی به خوشه‌های نهایی است. 
</div>

<div dir="rtl">
   الگوریتم‌های خوشه‌بندی مانند مخلوط‌های گوسی (Gaussian Mixture Models) می‌توانند به نمونه‌هایی که از دو خوشه با پراکندگی متفاوت پیش آمده‌اند، احتمالات نسبت دهند. این الگوریتم می‌تواند خوشه‌هایی با اشکال و اندازه‌های متفاوت را تشخیص دهد که ممکن است K-Means نتواند این کار را به خوبی انجام دهد.

از طرف دیگر، اگر برچسب‌های واقعی داده‌ها موجود باشد، می‌توان از روش‌های یادگیری با نظارت مانند کلاسیفیکیشن استفاده کرد. به طور خاص، می‌توان از الگوریتم‌های مانند Support Vector Machines (SVM) یا Random Forests برای دسته‌بندی داده‌های جدید استفاده کرد. این الگوریتم‌ها می‌توانند با استفاده از اطلاعات برچسب‌دار دقت بهتری را به دست آورند.
</div>